In [16]:
 training_data_path = 'path/to/training/folder'
testing_data_path = 'path/to/testing/folder'

In [30]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

from tensorflow.keras.applications import DenseNet121
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout,MaxPooling2D , Conv2D,Flatten
from tensorflow.keras.models import Sequential



from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

In [32]:
train_dir = "/content/Train"
test_dir = "/content/Test"


SEED = 12
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 128
EPOCHS = 5
LR = 0.00003

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of the ImageDataGenerator
data_generator = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    rotation_range=20,  # Randomly rotate images by 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20% of the width
    height_shift_range=0.2,  # Randomly shift images vertically by 20% of the height
    shear_range=0.2,  # Apply shear transformation with a shear intensity of 20%
    zoom_range=0.2,  # Randomly zoom images by up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    vertical_flip=False  # Do not flip images vertically
)

# Load the training data from the directory using the data generator
train_data = data_generator.flow_from_directory(
    directory='path/to/train_data_directory',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=True
)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the parameters
data_directory = 'path/to/data/directory'
batch_size = 64
target_size = (224, 224)
learning_rate = 0.001
seed = 42

# Create an instance of the ImageDataGenerator
data_generator = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
)

# Load the training data from the directory using the data generator
train_data = data_generator.flow_from_directory(
    directory=data_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=seed
)

# Configure learning rate and seed
learning_rate = 0.001
seed = 42

# Set the random seed for reproducibility
np.random.seed(seed)
tf.random.set_seed(seed)

# Create and compile your deep learning model
model = create_model()  # Replace create_model() with your actual model creation code
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train your model
model.fit(train_data, epochs=10)  # Replace epochs=10 with the desired number of training epochs


In [ ]:
import os

crimes = {}
train = test = 0

# Assuming crime_types is a list of different crime types
crime_types = ['burglary', 'robbery', 'assault']

for clss in crime_types:
    train_dir = 'path/to/train_data_directory'
    test_dir = 'path/to/test_data_directory'
    
    num = len(os.listdir(os.path.join(train_dir, clss)))
    train += num
    
    test += len(os.listdir(os.path.join(test_dir, clss)))
    
    crimes[clss] = num

print("Total training samples:", train)
print("Total testing samples:", test)
print("Crimes dictionary:", crimes)


In [ ]:
import matplotlib.pyplot as plt

# Assuming train and test are the variables containing the counts
train = 1000
test = 500

# Pie chart data
labels = ['Train', 'Test']
sizes = [train, test]
colors = ['#FFA500', '#008000']  # Orange and green

# Create a figure with specified size
plt.figure(figsize=(8, 5))

# Generate the pie chart
plt.pie(sizes, autopct='%.1f%%', explode=[0.1, 0], labels=labels, pctdistance=0.5, colors=colors)

# Add a title to the pie chart
plt.title("Train and Test Images", fontsize=18)

# Display the chart
plt.show()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Specify the directory where the images are located
directory = '/path/to/directory'

# Specify the class labels for the images
labels = ['class1', 'class2', 'class3']

# Specify the batch size
batch_size = 32

# Specify the image size to which the images should be resized
image_size = (224, 224)  # Example size

# Create an instance of the ImageDataGenerator
datagen = ImageDataGenerator()

# Generate the data generator object
data_generator = datagen.flow_from_directory(
    directory=directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Set to 'binary' for binary classification
    classes=labels
)


In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def create_densenet_model(input_shape, include_top=True, weights='imagenet', threshold=149):
    base_model = DenseNet121(include_top=include_top, weights=weights, input_shape=input_shape)
    
    # Set the threshold for the layers to be frozen
    for layer in base_model.layers:
        if int(layer.name.split('_')[-1]) < threshold:
            layer.trainable = False
    
    # Add custom layers on top of the base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)  # Replace `num_classes` with the number of output classes
    
    # Create the final model
    model = Model(inputs=base_model.input, outputs=output)
    
    return model


In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model

def create_transfer_model(input_shape, num_classes, include_top=False, weights='imagenet', freeze_layers=0):
    # Load the pre-trained DenseNet121 model
    base_model = DenseNet121(
        include_top=include_top,
        weights=weights,
        input_shape=input_shape
    )
    
    # Freeze the specified number of layers
    if freeze_layers > 0:
        for layer in base_model.layers[:freeze_layers]:
            layer.trainable = False
    
    # Add a new fully connected layer for classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    # Create the transfer learning model
    transfer_model = Model(inputs=base_model.input, outputs=predictions)
    
    return transfer_model


In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

def create_transfer_model(input_shape, num_classes, include_top=False, weights='imagenet', freeze_layers=0):
    # Load the pre-trained DenseNet121 model
    base_model = DenseNet121(
        include_top=include_top,
        weights=weights,
        input_shape=input_shape
    )
    
    # Freeze the specified number of layers
    if freeze_layers > 0:
        for layer in base_model.layers[:freeze_layers]:
            layer.trainable = False
    
    # Add a global average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    # Add dense layers
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    # Add the output layer
    predictions = Dense(num_classes, activation='softmax')(x)
    
    # Create the transfer learning model
    transfer_model = Model(inputs=base_model.input, outputs=predictions)
    
    return transfer_model


In [ ]:
from tensorflow.keras.optimizers import Adam

def configure_learning(model, learning_rate, loss_function, metrics):
    optimizer = Adam(learning_rate=learning_rate)  # Set the learning rate for the optimizer
    
    model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)


In [ ]:
learning_rate = 0.001
loss_function = 'categorical_crossentropy'
metrics = ['accuracy']

configure_learning(model, learning_rate, loss_function, metrics)


traning the mode 

In [ ]:
def train_model(model, train_dataset, validation_dataset, epochs):
    history = model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=epochs
    )
    return history


In [ ]:
epochs = 10
history = train_model(model, train_dataset, validation_dataset, epochs)


save the model


In [ ]:
def save_model(model, save_path):
    model.save(save_path)


In [ ]:
save_path = 'path/to/save/model.h5'
save_model(model, save_path)


In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model(save_path)


test the model

In [ ]:
def test_model(model, test_dataset):
    results = model.evaluate(test_dataset)
    return results


In [ ]:
test_results = test_model(model, test_dataset)


In [ ]:
predictions = model.predict(test_dataset)


 application building

building HDML pages

In [ ]:
from html import *

def create_html_page():
    # Create the HTML structure
    doc = html(
        head(
            title("My HTML Page")
        ),
        body(
            h1("Welcome to My HTML Page"),
            p("This is a sample HTML page created using Python."),
            p("Here are some example elements:")
        )
    )

    # Convert the HTML structure to a string
    html_string = str(doc)

    # Save the HTML string to a file
    with open("index.html", "w") as file:
        file.write(html_string)


build flack code 

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

if __name__ == '__main__':
    app.run()


In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>My Flask App</title>
</head>
<body>
    <h1>Welcome to My Flask App</h1>
    <p>This is a sample Flask application.</p>
</body>
</html>


run the application

In [ ]:
python app.py


In [ ]:
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


train the model on ibm

register for ibm cloud

Ideation Phase


Define The Problem Statement